In [8]:
import glob
import os
import re

In [9]:
seed_no = 4
dihedral = ['1_2_3_4'] 
fragment = ['1']

In [10]:
# process mol2 files to rename atom types as necessary

def process_mol2_file(input_file, output_file):
    """
    Processes a single .mol2 file to rename specific atom types while preserving formatting.

    Args:
        input_file (str): Path to the input .mol2 file.
        output_file (str): Path to save the modified .mol2 file.
    """
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    updated_lines = []
    in_atom_section = False

    for line in lines:
        if line.strip() == "@<TRIPOS>ATOM":
            in_atom_section = True
        elif line.strip() == "@<TRIPOS>BOND":
            in_atom_section = False

        if in_atom_section and line.strip() and not line.startswith("@<TRIPOS>"):

            # Only replace c3 atom type with CX atom type if the atom name is C3 (check mol2 file of staple to find the Calpha name)
            # if re.search(r'\sC3\s', line):
            #     line = re.sub(r'(\s)c3(\s)', r'\1CX\2', line)

            # change atom types of hydrogens to custom type hi
            # if re.search(r'\sH10\s', line):
            #     line = re.sub(r'(\s)hc(\s)', r'\1hi\2', line)
            # if re.search(r'\sH11\s', line):
            #     line = re.sub(r'(\s)hc(\s)', r'\1hi\2', line)
            
            line = re.sub(r'(\s)os(\s)', r'\1o \2', line)
            line = re.sub(r'(\s)n7(\s)', r'\1ns\2', line)
            line = re.sub(r'(\s)n(\s)', r'\1ns\2', line)
            line = re.sub(r'(\s)nd(\s)', r'\1nc\2', line)
            line = re.sub(r'(\s)cd(\s)', r'\1cc\2', line)

        updated_lines.append(line)

    with open(output_file, 'w') as outfile:
        outfile.writelines(updated_lines)


def batch_process_mol2_files(input_dir):
    """
    Batch processes all .mol2 files in a directory.

    Args:
        input_dir (str): Directory containing input .mol2 files.
    """
    for filename in os.listdir(input_dir):
        if filename.endswith(".mol2"):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(input_dir, f"{os.path.splitext(filename)[0]}_atomtypes_renamed.mol2")
            print(f"Processing {input_file} -> {output_file}")
            process_mol2_file(input_file, output_file)


In [11]:
# edit here to read parameters automatically (use other mol2 parameterisation scripts)
input_directory = "profiles_torsions/optimised_geometries/fragment%s/torsion%s/" % (fragment[0], dihedral[0])
batch_process_mol2_files(input_directory)

Processing profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_100_2_forward_resp_charges.mol2 -> profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_100_2_forward_resp_charges_atomtypes_renamed.mol2
Processing profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-50_1_forward_resp_charges.mol2 -> profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-50_1_forward_resp_charges_atomtypes_renamed.mol2
Processing profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-20_1_forward_resp_charges.mol2 -> profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-20_1_forward_resp_charges_atomtypes_renamed.mol2
Processing profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-140_2_backward_no_charges.mo

In [12]:
file_pattern = 'profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed%s_conf*_resp_charges_atomtypes_renamed.mol2' % (fragment, dihedral, dihedral, seed_no)

In [13]:
file_list = []

for frag in fragment:
    for dih in dihedral:
        file_pattern = 'profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed%s_conf*_resp_charges_atomtypes_renamed.mol2' % (frag, dih, dih, seed_no)

        file_list.extend(glob.glob(file_pattern))

file_list

['profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_130_2_backward_resp_charges_atomtypes_renamed.mol2',
 'profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-110_2_backward_resp_charges_atomtypes_renamed.mol2',
 'profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-30_1_forward_resp_charges_atomtypes_renamed.mol2',
 'profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_50_2_backward_resp_charges_atomtypes_renamed.mol2',
 'profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_-80_2_backward_resp_charges_atomtypes_renamed.mol2',
 'profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_80_1_forward_resp_charges_atomtypes_renamed.mol2',
 'profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe

In [14]:
for fle in file_list:
    tleapinput = open("tleap.in", "w")
    tleapinput.write(f"mol  = loadmol2 {fle} \n")
    tleapinput.write("loadamberparams click_staple_gaff2.frcmod \n") 
    tleapinput.write(f"saveamberparm mol {fle}.prmtop {fle}.inpcrd  \n")
    tleapinput.write("quit")
    tleapinput.close()

    os.system("tleap -f tleap.in")


-I: Adding /home/eva/anaconda3/envs/qm_resp_3.8/dat/leap/prep to search path.
-I: Adding /home/eva/anaconda3/envs/qm_resp_3.8/dat/leap/lib to search path.
-I: Adding /home/eva/anaconda3/envs/qm_resp_3.8/dat/leap/parm to search path.
-I: Adding /home/eva/anaconda3/envs/qm_resp_3.8/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
Loading Mol2 file: ./profiles_torsions/optimised_geometries/fragment1/torsion1_2_3_4/torsion1_2_3_4_seed4_conf0_dihe_130_2_backward_resp_charges_atomtypes_renamed.mol2
Reading MOLECULE named MOL								
Loading parameters: ./click_staple_gaff2.frcmod
Reading force field modification type file (frcmod)
Reading title:
Remark line goes here
Checking Unit.
Building topology.
Building atom parameters.
Building bond parameters.
Building angle parameters.
Building proper torsion parameters.
Building improper torsion parameters.
 total 2 improper torsions applied
Building H-Bond parameters.
Incorporating